<a href="https://www.kaggle.com/code/koenbotermans/kraken-obtaining-historical-trade-data-2022?scriptVersionId=178903882" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Kraken - Introduction with CCXT and obtaining historical trade data for 2022.
Are you intrigued by the potential of leveraging your data science skills to generate income through the creation of models on financial data? Have you ever pondered the possibility of crafting algorithmic trading strategies that could yield profitable results? Perhaps you're simply interested in conducting basic analyses of Bitcoin exchange data. Look no further. This series of notebooks commences with the fundamental step of acquiring historical trade data, setting the stage for a comprehensive exploration into the realm of financial analytics and algorithmic trading.

This notebook will be the first in a series where I work on obtaining and deriving insights from data from the [Kraken Exchange](https://www.kraken.com/), using the biggest and most convenient cryptocurrency toolbox in the world [CCXT](https://github.com/ccxt/ccxt). The output of this notebook will be all **historical trades for the trading pair BTC-USDT in the year 2022!**

After I give an introductory glimpse into (crypto) currency exchanges and how CCXT functions, I will give basic code snippets to show what you can do with CCXT.

## Why Kraken?
I choose to work with Kraken because it offers the most accessible platform for cryptocurrency trading and boasts the most welcoming and supportive community environment compared to other exchanges.

## Content
This notebook will give you answers, examples, and inspiration by answering the following questions:

1. Which exchanges are available for what services?
2. How to fetch the current price of the trading pair BTC-USDT? -> fetching ticker data.
3. How to fetch klines data from Kraken? Klines are also known as OHLCV (open high low close volume) data.
4. How to do some basic EDA on Kraken exchange data?
    - Plot the candlestick data with plotly
    - Plot the distribution of the (log-) returns of the Bitcoin.
    - Plot the distribution of the volume of the candlestick data.
5. How to access all the historical trades from 2022 on the pair: BTC-USDT.
6. How to export this data such that we can download them for further use?
7. How to convert these historical trades to klines?

## What is CCXT?

CCXT stands for CryptoCurrency Exchange Trading library. Thus one can use this library to access exchanges (like Kraken, Binance or Coinbase). Ofcourse, some of these exchanges (like Binance or Kraken) offer their own APIs or SDKs but CCXT simply combines them together to one simple libary. Which means that you can use the CCXT library to acces Binance, Kraken and Coinbase at the same time.

In [1]:
!pip install ccxt -q
!pip install plotly -q

In [2]:
import ccxt
from tqdm import tqdm
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# Which exchanges are available for what services?

The most common data you could derive from the exchanges 

1. Fetching **ticker**. A ticker for a symbol/market/pair (for example the pair BTC/USDT) is the most recent price payed for a symbol in a transaction. Fetching a ticker for a symbol is done with the `.fetch_ticker(symbol="BTC/USDT")` method.
2. Fetching the **ohlc** (open, high, low, close, volume). These are the open, high, low, close prices for a certain pair and the volume traded, for a given time period. This is done with the `fetch_ohlcv()` method.
3. Fetching the **order book** for a symbol. These are the currently requested trades in the order book for the symbol. This is done with the `.fetch_order_book()` method.
4. Fetching the **executed trades** for a symbol. This is done with the `.fetch_trades()` method.

Not every exchange offers all these services, so I created the following utility script that loops through all available exchanges and looks whether you can call the above methods without getting an error.

In [3]:
all_exchange_names = ccxt.exchanges
print("How many exchanges in ccxt:", len(all_exchange_names))
print("The first ten exchanges: ", all_exchange_names[:10])

How many exchanges in ccxt: 105
The first ten exchanges:  ['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx']


In [4]:
def derive_available_services(symbol: str ="BTC/USDT") -> pd.DataFrame:
    exchange_info = {}
    exchange_info["names"] = ccxt.exchanges
    exchange_info["available_ohlcv"] = []
    exchange_info["available_orderbook"] = []
    exchange_info["available_trades"] = []
    exchange_info["available_ticker"] = []

    for exchange_name in tqdm(exchange_info["names"]):
                
        #Initialize a exchange class
        exchange_class = getattr(ccxt, exchange_name)
        exchange = exchange_class()
        
        
        #Check whether tickers are available for this exchange
        try:
            exchange.fetch_ticker(symbol=symbol)
            exchange_info["available_ticker"].append("Exist")
        except Exception as inst:
            exchange_info["available_ticker"].append(type(inst))
            pass
        
        try:
            exchange.fetch_ohlcv(symbol=symbol)
            exchange_info["available_ohlcv"].append("Exist")
        except Exception as inst:
            exchange_info["available_ohlcv"].append(type(inst))
            pass
        
        try:
            exchange.fetch_order_book(symbol=symbol)
            exchange_info["available_orderbook"].append("Exist")
        except Exception as inst:
            exchange_info["available_orderbook"].append(type(inst))
            pass
        
        try:
            exchange.fetch_trades(symbol=symbol)
            exchange_info["available_trades"].append("Exist")
        except Exception as inst:
            exchange_info["available_trades"].append(type(inst))
    exchange_info_df = pd.DataFrame(exchange_info)
    return exchange_info_df.set_index(keys=["names"])

In [5]:
exchange_services_df = derive_available_services(symbol="BTC/USDT")

100%|██████████| 105/105 [05:32<00:00,  3.17s/it]


In [6]:
results = {}
results["column"] = []
results["count"] = []
for column in exchange_services_df.columns:
    results["count"].append( np.sum(exchange_services_df[column] == "Exist"))
    results["column"].append( column)

In [7]:
results

{'column': ['available_ohlcv',
  'available_orderbook',
  'available_trades',
  'available_ticker'],
 'count': [58, 70, 68, 71]}

In [8]:
exchange_services_df

,available_ohlcv,available_orderbook,available_trades,available_ticker
names,,,,
ace,<class 'ccxt.base.errors.AuthenticationError'>,<class 'ccxt.base.errors.ExchangeNotAvailable'>,<class 'ccxt.base.errors.NotSupported'>,Exist
alpaca,<class 'ccxt.base.errors.PermissionDenied'>,<class 'ccxt.base.errors.PermissionDenied'>,<class 'ccxt.base.errors.PermissionDenied'>,<class 'ccxt.base.errors.NotSupported'>
ascendex,Exist,Exist,Exist,Exist
bequant,Exist,Exist,Exist,Exist
bigone,Exist,Exist,Exist,Exist
...,...,...,...,...
woo,Exist,Exist,Exist,<class 'ccxt.base.errors.NotSupported'>
woofipro,<class 'ccxt.base.errors.BadSymbol'>,<class 'ccxt.base.errors.BadSymbol'>,<class 'ccxt.base.errors.BadSymbol'>,<class 'ccxt.base.errors.NotSupported'>
yobit,<class 'ccxt.base.errors.NotSupported'>,Exist,Exist,Exist


In [9]:
results_df = pd.DataFrame(results)
results_df

,column,count
0,available_ohlcv,58
1,available_orderbook,70
2,available_trades,68
3,available_ticker,71


In [10]:
import plotly.express as px
fig = px.bar(results, x='column', y='count')

fig.update_layout(
    title='Availability of Services',
    xaxis_title='Availability Services',
    yaxis_title='Count',
    legend_title='Services',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

## Exchange availability
Let's just print the results for clarity;

In [11]:
print(f"This means that from the {len(results_df)} available exchanges in the toolbox;")
print(f"{results_df[results_df['column'] == 'available_ohlcv']['count'].values[0]}/{len(exchange_services_df)} have public available ohlcv data.")
print(f"{results_df[results_df['column'] == 'available_orderbook']['count'].values[0]}/{len(exchange_services_df)} have public available orderbook data.")
print(f"{results_df[results_df['column'] == 'available_trades']['count'].values[0]}/{len(exchange_services_df)} have public available trade data.")
print(f"{results_df[results_df['column'] == 'available_ticker']['count'].values[0]}/{len(exchange_services_df)} have public available ticker data.")

This means that from the 4 available exchanges in the toolbox;
58/105 have public available ohlcv data.
70/105 have public available orderbook data.
68/105 have public available trade data.
71/105 have public available ticker data.


In [12]:
ohlcv_column_names = ["Timestamp", "Open", "High", "Low", "Close", "Volume"]
symbol = "BTC/USDT"
timeframe = "1h"

## Select the karken exchange.

In [13]:
kraken_exchange = ccxt.kraken()

# How to fetch the current price of the traiding pair BTC-USDT?  ->  fetching ticker data.

Ticker data means the most recent price for a certain symbol. Fetching a ticker is the most basic opperation one can do on an exchange.

In [14]:
kraken_exchange.fetch_ticker(symbol=symbol)

{'symbol': 'BTC/USDT',
 'timestamp': None,
 'datetime': None,
 'high': 71887.8,
 'low': 66749.3,
 'bid': 70939.0,
 'bidVolume': None,
 'ask': 70939.1,
 'askVolume': None,
 'vwap': 69652.48915,
 'open': 71430.0,
 'close': 70922.1,
 'last': 70922.1,
 'previousClose': None,
 'change': -507.9,
 'percentage': -0.7110457790844184,
 'average': 71176.05,
 'baseVolume': 505.62122364,
 'quoteVolume': 35217776.79359482,
 'info': {'a': ['70939.10000', '1', '1.000'],
  'b': ['70939.00000', '1', '1.000'],
  'c': ['70922.10000', '0.01386232'],
  'v': ['162.77120329', '505.62122364'],
  'p': ['71095.68319', '69652.48915'],
  't': ['4530', '15039'],
  'l': ['70534.80000', '66749.30000'],
  'h': ['71887.80000', '71887.80000'],
  'o': '71430.00000'}}

# How to fetch klines data from Kraken?.


OHLCV (stands for: Open High Low Close Volume, often called klines) show the price movements and trading activity within a specific time frame, aiding in technical analysis and decision-making by traders.

Many plotting libraries have convenient ways to plot them, 

In [15]:
btc_ohlcv_response = kraken_exchange.fetch_ohlcv(symbol=symbol, timeframe=timeframe)
btc_ohlcv_df = pd.DataFrame(btc_ohlcv_response, columns=ohlcv_column_names)
btc_ohlcv_df["Timestamp"] = pd.to_datetime(btc_ohlcv_df["Timestamp"], unit="ms")
btc_ohlcv_df.head()

,Timestamp,Open,High,Low,Close,Volume
0,2024-04-21 09:00:00,64926.5,64970.3,64829.7,64890.9,3.913295
1,2024-04-21 10:00:00,64886.7,64983.9,64829.6,64963.5,2.030665
2,2024-04-21 11:00:00,64959.7,65382.0,64953.9,65238.1,3.869091
3,2024-04-21 12:00:00,65237.3,65354.4,64963.0,65015.0,14.638985
4,2024-04-21 13:00:00,64999.8,65213.7,64829.6,64881.9,2.060332


In [16]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_ohlcv_df['Timestamp'],
                open=btc_ohlcv_df['Open'],
                high=btc_ohlcv_df['High'],
                low=btc_ohlcv_df['Low'],
                close=btc_ohlcv_df['Close'])])
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc_ohlcv_df['Timestamp'],
                open=btc_ohlcv_df['Open'],
                high=btc_ohlcv_df['High'],
                low=btc_ohlcv_df['Low'],
                close=btc_ohlcv_df['Close'])])

fig.update_layout(
    title='BTC Candlestick Chart - Kraken Exchange',
    xaxis_title='Timestamp',
    yaxis_title='Price',
    legend_title='Price',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)
fig.show()

# How to do some basic EDA on these klines data?

Performing basic Exploratory Data Analysis (EDA) on klines data in financial markets is essential for gaining insights into the underlying patterns and characteristics of the price movements. By analyzing key metrics such as open, high, low, close prices, as well as volume data, EDA helps identify trends, volatility, and potential trading opportunities. It also aids in understanding the distribution of price movements, detecting outliers, and validating assumptions, ultimately informing decision-making processes for traders and investors. Which is exactly what we will be doing in this section of the notebook!


## Visualizing historical returns

To aid in interprenting the data, let's visualize the returns. This might give us some insight in the historical volatility and the performance of this traiding pair.

In [17]:
btc_ohlcv_df["return"] = btc_ohlcv_df["Close"]/btc_ohlcv_df["Open"]

In [18]:

import plotly.figure_factory as ff

fig = ff.create_distplot([btc_ohlcv_df["return"]], ["return"])
fig.update_xaxes(range=[btc_ohlcv_df["return"].min()*0.99, btc_ohlcv_df["return"].max()*1.01])

fig.update_layout(
    title='Distribution of Bitcoin Returns',
    xaxis_title='Returns',
    yaxis_title='Frequency',
    legend_title='Returns',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)


fig.show()


## QQ plot on the returns
> A Q-Q plot (a quantile-quantile plot) is a graphical method for comparing two probability distributions by plotting their quantiles against each other. If the two distributions being compared are similar, the points in the Q-Q plot will approximately lie on the identity line. 
[Wiki](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot)



In [19]:
from scipy import stats
qq = stats.probplot(btc_ohlcv_df["return"], dist='norm', sparams=(1))
x = np.array([qq[0][0][0], qq[0][0][-1]])

fig = go.Figure()
fig.add_scatter(x=qq[0][0], y=qq[0][1], mode='markers')
fig.add_scatter(x=x, y=qq[1][1] + qq[1][0]*x, mode='lines')

fig.update_layout(
    title='QQ Plot of Bitcoin Returns',
    xaxis_title='Theoretical Quantiles',
    yaxis_title='Sample Quantiles',
    showlegend=False
)

fig.show()

In [20]:
btc_ohlcv_df["log_return"] = np.log(btc_ohlcv_df["return"])

fig = ff.create_distplot([btc_ohlcv_df["log_return"]], ["log_return"])
fig.update_xaxes(range=[btc_ohlcv_df["log_return"].min()*0.99, btc_ohlcv_df["log_return"].max()*1.01])
fig.update_layout(
    title='Distribution of Bitcoin Log-Returns',
    xaxis_title='Log-Returns',
    yaxis_title='Frequency',
    legend_title='Returns',
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()


In [21]:
import numpy as np 
import scipy.stats as stats

measurements = np.random.normal(loc = 20, scale = 5, size=100)   
qq_plot_data = stats.probplot(btc_ohlcv_df["log_return"], dist="norm")
x = np.array([qq_plot_data[0][0][0], qq_plot_data[0][0][-1]])

fig = go.Figure()
fig.add_scatter(x=qq_plot_data[0][0], y=qq_plot_data[0][1], mode='markers')
fig.add_scatter(x=x, y=qq_plot_data[1][1] + qq_plot_data[1][0]*x, mode='lines')
fig.layout.update(showlegend=False)
fig.show()

# How to access all the historical trades from 2022 on the pair: BTC-USDT.
Unfortunatly, the Kraken API only allows to obtain 720 data endpoints for a given requests [link](https://support.kraken.com/hc/en-us/articles/218198197-How-to-retrieve-historical-time-and-sales-trading-history-using-the-REST-API-Trades-endpoint-)....

Fortunaly, there is an easy way arount this, by making our own ohlcv candlestick data. Which can be done by aggregating the trades data. This is done in the cells below.

In [22]:
def fetch_trades(
    exchange: ccxt.Exchange, 
    symbol: str = "BTC/USDT", 
    start_time: str = "2022-01-01 00:00:00", 
    until_time: str = "2023-01-01 00:00:00", 
    verbose: bool = False) -> pd.DataFrame:
    """Given an exchange, start_time, and end_time, this function fetches all available trades and returns 
    them in a pd.DataFrame.

    Args:
        exchange (ccxt.Exchange): The ccxt exchange which will be called.
        symbol (str, optional): The symbol which represents the market of which trades are fetched. Defaults to "BTC/USDT".
        start_time (_type_, optional): First time of the first trades. Defaults to "2022-01-01 00:00:00".
        until_time (_type_, optional): Time of the last trade to fetch. Defaults to "2023-01-01 00:00:00".
        verbose (bool, optional): Whether to print the progress. Defaults to False.

    Returns:
        pd.DataFrame: Contains all trades of market symbol, between start_time 
    """
    
    most_recent_trade_timestamp = exchange.parse8601(start_time)
    new_most_recent_trade_timestamp = None
    historical_trades = exchange.fetch_trades(symbol=symbol, since=most_recent_trade_timestamp)
    
    while pd.to_datetime(most_recent_trade_timestamp, unit="ms") < pd.to_datetime(until_time):
        new_trades = exchange.fetch_trades(symbol=symbol, since=most_recent_trade_timestamp)
        new_most_recent_trade_timestamp = new_trades[-1]["timestamp"]
        
        if new_most_recent_trade_timestamp == most_recent_trade_timestamp:
            break
        most_recent_trade_timestamp = new_most_recent_trade_timestamp
        historical_trades.extend(new_trades)
        if verbose:
            print(f"{pd.to_datetime(most_recent_trade_timestamp, unit='ms')}")

    
    historical_trades_df = pd.DataFrame(historical_trades)
    historical_trades_df["timestamp"] = pd.to_datetime(historical_trades_df["timestamp"], unit="ms")
    historical_trades_df = historical_trades_df.set_index(keys=["timestamp"])

    return historical_trades_df
historical_trades_df =fetch_trades(exchange=kraken_exchange)


In [23]:
historical_trades_df.to_csv("kraken_2022_historical_trades_btcusdt.csv")

# How to convert these historical trades to klines?

So we couldn't fetch the historical klines data, so we fetched all the historical trades. How do we convert those to klines you might ask? Well, I will explain that in the next notebook!